<a href="https://colab.research.google.com/github/lake-thomas/spurge-temporal-cnn/blob/main/Temporal_CNN_Leafy_Spurge_Generate_Training_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This is a working Python notebook to implement Google Earth Engine <> TensorFlow for mapping invasive plant species from a time-series of Landsat imagery. In this example, the inputs are invasive species occurrence records from public databases. The model uses 1D-Conv layers in a temporal CNN framework.

In [ ]:
# Cloud Authentication 
# Required When Using Default Google Cloud (i.e. Not Using a Hosted VM Runtime Environment)

#Connect to hosted VM https://console.cloud.google.com/marketplace/product/colab-marketplace-image-public/colab?project=pacific-engine-346519

from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()


In [ ]:
#Mount Google Drive for CSV reading
from google.colab import drive
drive.mount('/content/drive')

#Used to export to google cloud
from google.cloud import storage
import os

# Only need this if you're running on GCE
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/drive/MyDrive/Colab Notebooks/pacific-engine-346519-8368a64310cd.json'



Mounted at /content/drive


In [ ]:
#Ignore Warnings and Errors
!pip install geemap
import geemap #advanced python function for GEE
!pip install geopandas
import geopandas #Pandas library to handle geospatial data
!pip install fsspec
import fsspec # file system specification
!pip install gcsfs
import gcsfs #google cloud file system

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pprint
import time
from functools import reduce

In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)


2.8.0


#Load Functions
Functions to process Landsat imagery

In [ ]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)


#Cloud Mask: https://gis.stackexchange.com/questions/274048/apply-cloud-mask-to-landsat-imagery-in-google-earth-engine-python-api
def getQABits(image, start, end, mascara): 
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)


#Saturated band Mask: https://gis.stackexchange.com/questions/363929/how-to-apply-a-bitmask-for-radiometric-saturation-qa-in-a-image-collection-eart
def bitwiseExtract(value, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return value.rightShift(fromBit).bitwiseAnd(mask)


#Function to mask out cloudy and saturated pixels and harmonize between Landsat 5/7/8 imagery 
def maskQuality(image):
    # Select the QA band.
    QA = image.select('QA_PIXEL')
    # Get the internal_cloud_algorithm_flag bit.
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    #  var cloud_confidence = getQABits(QA,6,7,  'cloud_confidence')
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    #var cirrus_detected2 = getQABits(QA,8,8,  'cirrus_detected2')
    #Return an image masking out cloudy areas.
    QA_radsat = image.select('QA_RADSAT')
    saturated = bitwiseExtract(QA_radsat, 1, 7)

    #Apply the scaling factors to the appropriate bands.
    def getFactorImg(factorNames):
      factorList = image.toDictionary().select(factorNames).values()
      return ee.Image.constant(factorList)

    scaleImg = getFactorImg(['REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'])

    offsetImg = getFactorImg(['REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'])
    
    scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg)

    #Replace original bands with scaled bands and apply masks.
    return image.addBands(scaled, None, True).updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0).updateMask(saturated.eq(0))))


# Selects and renames bands of interest for Landsat OLI.
def renameOli(img):
  return img.select(
    ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL', 'QA_RADSAT'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'QA_PIXEL', 'QA_RADSAT'])


# Selects and renames bands of interest for TM/ETM+.
def renameEtm(img):
  return img.select(
    ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL', 'QA_RADSAT'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'QA_PIXEL', 'QA_RADSAT'])


# Adding a NDVI band
def addNDVI(image):
  ndvi = image.normalizedDifference(['NIR', 'Red']).rename('NDVI')
  return image.addBands([ndvi])


def mapDates(image):
  date = ee.Date(image.get('system:time_start')).format("YYYY-MM-dd")
  return image.addBands([date])

# Prepares (renames) OLI images.
def prepOli(img):
  img = renameOli(img)
  return img


# Prepares (renames) TM/ETM+ images.
def prepEtm(img):
  orig = img
  img = renameEtm(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()))


# Selects and renames bands of interest for TM/ETM+.
def renameImageBands_TM(img, year, season):
  return img.select(
      ['Blue_median', 'Green_median', 'Red_median', 'NIR_median', 
       'SWIR1_median', 'SWIR2_median', 'QA_PIXEL_median', 'QA_RADSAT_median', 'NDVI_median'],
      ['Blue'+str(season)+str(year), 'Green'+str(season)+str(year), 'Red'+str(season)+str(year), 'NIR'+str(season)+str(year),
       'SWIR1'+str(season)+str(year), 'SWIR2'+str(season)+str(year), 'QA_PIXEL'+str(season)+str(year), 'QA_RADSAT'+str(season)+str(year), 'NDVI'+str(season)+str(year)])

# Selects and renames bands of interest for TM/ETM+.
def renameImageBands_ETMOLI(img, year, season):
  return img.select(
      ['Blue_median_median', 'Green_median_median', 'Red_median_median', 'NIR_median_median', 
       'SWIR1_median_median', 'SWIR2_median_median', 'QA_PIXEL_median_median', 'QA_RADSAT_median_median', 'NDVI_median_median'],
      ['Blue'+str(season)+str(year), 'Green'+str(season)+str(year), 'Red'+str(season)+str(year), 'NIR'+str(season)+str(year),
       'SWIR1'+str(season)+str(year), 'SWIR2'+str(season)+str(year), 'QA_PIXEL'+str(season)+str(year), 'QA_RADSAT'+str(season)+str(year), 'NDVI'+str(season)+str(year)])


def getLandsatMosaicFromPoints(year, points):
  '''
  #Time-series extraction developed from
  #https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair#combine_dataframes  

  '''

  #if Year is between 1985 and 1999 use Landsat 5 TM imagery
  if 1985 <= year <= 1999:

    tmColMarchApril = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMarchApril = renameImageBands_TM(tmColMarchApril, year, 'MarchApril')

    tmColMayJune = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMayJune = renameImageBands_TM(tmColMayJune, year, 'MayJune')

    tmColJulyAug = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColJulyAug = renameImageBands_TM(tmColJulyAug, year, 'JulyAug')

    landsat5ImageCol = [tmColMarchApril, tmColMayJune, tmColJulyAug]
    return landsat5ImageCol

  #if Year is between 2000 and 2012 use mosaic from Landsat 5 TM and Landsat 7 ETM imagery
  elif 2000 <= year <= 2012:

    etmColMarchApril = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMarchApril = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    MarchApril = ee.ImageCollection([etmColMarchApril, tmColMarchApril])

    etmColMarchApril = MarchApril.reduce('median')

    etmColMarchApril = renameImageBands_ETMOLI(etmColMarchApril, year, 'MarchApril')

    etmColMayJune = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMayJune = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    MayJune = ee.ImageCollection([etmColMayJune, tmColMayJune])

    etmColMayJune = MayJune.reduce('median')

    etmColMayJune = renameImageBands_ETMOLI(etmColMayJune, year, 'MayJune')

    etmColJulyAug = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColJulyAug = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    JulyAug = ee.ImageCollection([etmColJulyAug, tmColJulyAug])

    etmColJulyAug = JulyAug.reduce('median')

    etmColJulyAug = renameImageBands_ETMOLI(etmColJulyAug, year, 'JulyAug')

    landsat5_7ImageCol = [etmColMarchApril, etmColMayJune, etmColJulyAug]
    return landsat5_7ImageCol

  #if Year is between 2013 and 2020 use mosaic from Landsat 7 ETM and Landsat 8 OLI imagery
  elif 2013 <= year <= 2020:

    etmColMarchApril = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    oliColMarchApril = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepOli) \
      .map(addNDVI) \
      .reduce('median')

    MarchApril = ee.ImageCollection([etmColMarchApril, oliColMarchApril])

    etmColMarchApril = MarchApril.reduce('median')

    etmColMarchApril = renameImageBands_ETMOLI(etmColMarchApril, year, 'MarchApril')

    etmColMayJune = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    oliColMayJune = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepOli) \
      .map(addNDVI) \
      .reduce('median')

    MayJune = ee.ImageCollection([etmColMayJune, oliColMayJune])

    etmColMayJune = MayJune.reduce('median')

    etmColMayJune = renameImageBands_ETMOLI(etmColMayJune, year, 'MayJune')

    etmColJulyAug = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median') \

    oliColJulyAug = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepOli) \
      .map(addNDVI) \
      .reduce('median')

    JulyAug = ee.ImageCollection([etmColJulyAug, oliColJulyAug])

    etmColJulyAug = JulyAug.reduce('median')

    etmColJulyAug = renameImageBands_ETMOLI(etmColJulyAug, year, 'JulyAug')

    landsat7_8ImageCol = [etmColMarchApril, etmColMayJune, etmColJulyAug]
    return landsat7_8ImageCol



def sampleImagestoDataFrame(listofEEImages):
    '''
    Function takes in a list of three images from a Landsat imagery year (T1, T2, T3)
    Returns a merged pandas dataframe of dimensions (rows/samples x bands) ordered from t-1, t, t+1
    '''
    image1 = listofEEImages[0]
    image2 = listofEEImages[1]
    image3 = listofEEImages[2]

    image1_fc = image1.sampleRegions(collection=newpts, properties=['class'], scale=30)
    image2_fc = image2.sampleRegions(collection=newpts, properties=['class'], scale=30)
    image3_fc = image3.sampleRegions(collection=newpts, properties=['class'], scale=30)

    image1_db_dict = fc_to_dict(image1_fc).getInfo()
    image2_db_dict = fc_to_dict(image2_fc).getInfo()
    image3_db_dict = fc_to_dict(image3_fc).getInfo()

    image1_df = pd.DataFrame(image1_db_dict)
    image2_df = pd.DataFrame(image2_db_dict)
    image3_df = pd.DataFrame(image3_db_dict)

    data_frames = [image1_df, image2_df, image3_df]

    df_merged = reduce(lambda left,right: pd.merge(left, right, on='system:index', how='outer'), data_frames).fillna(np.nan)

    df_merged_dropna = df_merged.dropna(axis=0, how = 'any')

    return df_merged_dropna



#Create Bounding Box Moving Windows Across Study Region

In [ ]:


#Generate Bounding Box Coordinate List for Study Region ###
#Starting position of bounding box
XY_topLeft = [-116.976099, 48.904682]
XY_topRight = [-115.976099, 48.904682]
XY_bottomLeft = [-116.976099, 47.904682]
XY_bottomRight = [-115.976099, 47.904682]

lon_range = 31 #study area spans 31 deg lon
lat_range = 12 #study area spans 12 deg lat

stepSize = 1 #step by 1 degree of long/latitude


def sliding_window(longitude_range, latitude_range, stepSize_box):
    lon_list = []
    lat_list = []
    for lon in range(0, longitude_range, stepSize_box):
      for lat in range(0, latitude_range,stepSize_box):
        lon_list.append(lon)
        lat_list.append(lat)
    
    return(lon_list, lat_list)

def bbox(longitude_range, latitude_range, stepSize_box, topLeft_coord, topRight_coord, bottomLeft_coord, bottomRight_coord):
  #Creates a sliding window across the lat/long range
  #Returns a list of all lat/long boxes to sample 
     
  lon_list, lat_list = sliding_window(longitude_range, latitude_range, stepSize_box) #Generates two lists: one of longitude[0-31] and one of latitude [0-12] defining study region

  #for w in range(len(windows[0])):
  #  w_lon = windows[0][w]
  #  w_lat = windows[1][w]
  #  #print(w_lon, w_lat)

  #Top Left Coordinates for BBox
  lon_list_X_topLeft = [x + topLeft_coord[0] for x in lon_list]
  lat_list_Y_topLeft = [abs(x - topLeft_coord[1]) for x in lat_list]
  XY_topLeft_list = list(zip(lon_list_X_topLeft, lat_list_Y_topLeft))

  #Bottom Left Coordinates for BBox
  lon_list_X_bottomLeft = [x + bottomLeft_coord[0] for x in lon_list]
  lat_list_Y_bottomLeft = [abs(x - bottomLeft_coord[1]) for x in lat_list]
  XY_bottomLeft_list = list(zip(lon_list_X_bottomLeft, lat_list_Y_bottomLeft))

  #Top Right Coordinates for BBox
  lon_list_X_topRight = [x + topRight_coord[0] for x in lon_list]
  lat_list_Y_topRight = [abs(x - topRight_coord[1]) for x in lat_list]
  XY_topRight_list = list(zip(lon_list_X_topRight, lat_list_Y_topRight))

  #Bottom Right Coordinates for BBox
  lon_list_X_bottomRight = [x + bottomRight_coord[0] for x in lon_list]
  lat_list_Y_bottomRight = [abs(x - bottomRight_coord[1]) for x in lat_list]
  XY_bottomRight_list = list(zip(lon_list_X_bottomRight, lat_list_Y_bottomRight))

  ### Bounding Box Coordinate List
  bbox = list(zip(XY_topLeft_list, XY_bottomLeft_list, XY_topRight_list, XY_bottomRight_list))

  return bbox


bbox_windows = bbox(lon_range, lat_range, stepSize, XY_topLeft, XY_topRight, XY_bottomLeft, XY_bottomRight)


# Basic Workflow to Generate Training Datasets

Iteratively generate bounding box arcross study area. Within each bounding box, extract points with labeled land cover values (including leafy spurge) and Landsat imagery. 

In [ ]:
#Training points for leafy spurge & land cover classes (defines extent of landsat imagery)

#Load 1m training points sampled from 2019 NLCD and leafy spurge from 2018-2019-2020
pts = ee.FeatureCollection('projects/pacific-engine-346519/assets/spurge_landcover_nlcd2019_onemillionpoints_april2022')

# Define export for feature class assets
outputBucket = 'landcover_samples_nlcd2019_onemillionpoints'
# Make sure the bucket exists.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + outputBucket) 
  else 'Output Cloud Storage bucket does not exist.')

#define years to sample data (corresponds to satellite image year)
years = [2018, 2019, 2020]

#Moving Bounding Box Loop to Generate Sample Points
for i in range(0, 2):
#for i in range(3): testing only
  
  print(i)

  # Define Bounding Box
  bbox = bbox_windows[i]
  print(bbox)

   # Filter points based on AOI
  aoi = ee.Geometry.Polygon(bbox)

  #Apply Filter
  newpts = pts.filterBounds(aoi)
  
  #How many points?
  count = newpts.size() #returns an EE.Number object that we need to convert to an interger
  num_points = int(count.getInfo())
  print('Number of Points within AOI (Count): ', str(count.getInfo())+'\n')
  
  if num_points > 0:

    #Sample points given a year, here we want points sampled from three years of Landsat imagery centered on 2019 
    LandsatCol_year0 = getLandsatMosaicFromPoints(years[0], newpts) #output is a list of length 3, with three EEimages corresponding to three seasons in a year
    LandsatCol_year1 = getLandsatMosaicFromPoints(years[1], newpts)
    LandsatCol_year2 = getLandsatMosaicFromPoints(years[2], newpts)

    #Convert a LandsatCol_year Image List to a Merged Pandas DataFrame
    year0_df = sampleImagestoDataFrame(LandsatCol_year0)
    year1_df = sampleImagestoDataFrame(LandsatCol_year1)
    year2_df = sampleImagestoDataFrame(LandsatCol_year2)

    #Merge DFs and Drop NAs
    df_merged = reduce(lambda left,right: pd.merge(left, right, on='system:index', how='outer'), [year0_df, year1_df, year2_df]).fillna(np.nan)

    df_merged_dropna = df_merged.dropna(axis=0, how = 'any')

    #print(year0_df['class'].value_counts(), year1_df['class'].value_counts(), year2_df['class'].value_counts(), df_merged['class'].value_counts(), df_merged_dropna['class'].value_counts())

    #display(df_merged_dropna)


    #client = storage.Client()
    #bucket = client.get_bucket(outputBucket)
      
    #bucket.blob('nlcd2019/Landsat_2018-2020_Samples_Test_' + str(i) + '.csv').upload_from_string(df_merged_dropna.to_csv(), 'text/csv')
    #df_merged_dropna.to_csv('/content/drive/MyDrive/Colab Notebooks/Temporal CNN Exports CSV/Landsat_2018-2020_Samples_Test_' + str(i) + '.csv')
  
  
  


# Function to Gather Landsat ImageCollection Mosaic from Date/Points Input


In [ ]:


#Function to Gather Landsat ImageCollection Mosaic from Date/Points Input


# Selects and renames bands of interest for TM/ETM+.
def renameImageBands(img, year, season):
  return img.select(
      ['Blue_median', 'Green_median', 'Red_median', 'NIR_median', 
       'SWIR1_median', 'SWIR2_median', 'QA_PIXEL_median', 'QA_RADSAT_median', 'NDVI_median'],
      ['Blue'+str(season)+str(year), 'Green'+str(season)+str(year), 'Red'+str(season)+str(year), 'NIR'+str(season)+str(year),
       'SWIR1'+str(season)+str(year), 'SWIR2'+str(season)+str(year), 'QA_PIXEL'+str(season)+str(year), 'QA_RADSAT'+str(season)+str(year), 'NDVI'+str(season)+str(year)])


def getLandsatMosaicFromPoints(year, points):
  '''
  #Time-series extraction developed from
  #https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair#combine_dataframes  

  '''

  #if Year is between 1985 and 1999 use Landsat 5 TM imagery
  if 1985 <= year <= 1999:

    tmColMarchApril = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMarchApril = renameImageBands(tmColMarchApril, year, 'MarchApril')

    tmColMayJune = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMayJune = renameImageBands(tmColMayJune, year, 'MayJune')

    tmColJulyAug = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColJulyAug = renameImageBands(tmColJulyAug, year, 'JulyAug')

    landsat5ImageCol = [tmColMarchApril, tmColMayJune, tmColJulyAug]
    return landsat5ImageCol

  #if Year is between 2000 and 2012 use mosaic from Landsat 5 TM and Landsat 7 ETM imagery
  if 2000 <= year <= 2012:

    etmColMarchApril = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMarchApril = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    MarchApril = ee.ImageCollection([etmColMarchApril, tmColMarchApril])

    etmColMarchApril = MarchApril.reduce('median')

    etmColMayJune = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColMayJune = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    MayJune = ee.ImageCollection([etmColMayJune, tmColMayJune])

    etmColMayJune = MayJune.reduce('median')

    etmColJulyAug = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    tmColJulyAug = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    JulyAug = ee.ImageCollection([etmColJulyAug, tmColJulyAug])

    etmColJulyAug = JulyAug.reduce('median')

    landsat5_7ImageCol = ee.ImageCollection([etmColMarchApril, etmColMayJune, etmColJulyAug])
    return landsat5_7ImageCol

  #if Year is between 2013 and 2020 use mosaic from Landsat 7 ETM and Landsat 8 OLI imagery
  if 2013 <= year <= 2020:

    etmColMarchApril = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    oliColMarchApril = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
      .filterDate('{}-03-01'.format(year), '{}-04-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepOli) \
      .map(addNDVI) \
      .reduce('median')

    MarchApril = ee.ImageCollection([etmColMarchApril, oliColMarchApril])

    etmColMarchApril = MarchApril.reduce('median')

    etmColMayJune = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median')

    oliColMayJune = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
      .filterDate('{}-05-01'.format(year), '{}-06-30'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepOli) \
      .map(addNDVI) \
      .reduce('median')

    MayJune = ee.ImageCollection([etmColMayJune, oliColMayJune])

    etmColMayJune = MayJune.reduce('median')

    etmColJulyAug = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepEtm) \
      .map(addNDVI) \
      .reduce('median') \

    oliColJulyAug = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
      .filterDate('{}-07-01'.format(year), '{}-08-31'.format(year)) \
      .filterBounds(points) \
      .map(maskQuality) \
      .map(prepOli) \
      .map(addNDVI) \
      .reduce('median')

    JulyAug = ee.ImageCollection([etmColJulyAug, oliColJulyAug])

    etmColJulyAug = JulyAug.reduce('median')

    landsat7_8ImageCol = ee.ImageCollection([etmColMarchApril, etmColMayJune, etmColJulyAug])
    return landsat7_8ImageCol


year = 1987

landsatCol = getLandsatMosaicFromPoints(year, newpts)

image1 = landsatCol[0]
image2 = landsatCol[1]
image3 = landsatCol[2]

image1_fc = image1.sampleRegions(collection=newpts, properties=['class'], scale=30)
image2_fc = image2.sampleRegions(collection=newpts, properties=['class'], scale=30)
image3_fc = image3.sampleRegions(collection=newpts, properties=['class'], scale=30)


# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

image1_db_dict = fc_to_dict(image1_fc).getInfo()
image2_db_dict = fc_to_dict(image2_fc).getInfo()
image3_db_dict = fc_to_dict(image3_fc).getInfo()

image1_df = pd.DataFrame(image1_db_dict)
image2_df = pd.DataFrame(image2_db_dict)
image3_df = pd.DataFrame(image3_db_dict)

display(image1_df)

data_frames = [image1_df, image2_df, image3_df]


from functools import reduce
df_merged = reduce(lambda left,right: pd.merge(left, right, on='system:index', how='outer'), data_frames).fillna(np.nan)
display(df_merged)

df_merged_dropna = df_merged.dropna(axis=0, how = 'any')
#display(df_merged_dropna)


df_merged_removeQA = df_merged_dropna.drop(['QA_PIXEL_mean_1', 'QA_RADSAT_mean_1', 'QA_PIXEL_mean_2', 'QA_RADSAT_mean_2', 'QA_PIXEL_mean_3', 'QA_RADSAT_mean_3',
                                     'QA_PIXEL_mean_4', 'QA_RADSAT_mean_4', 'QA_PIXEL_mean_5', 'QA_RADSAT_mean_5', 'QA_PIXEL_mean_6', 'QA_RADSAT_mean_6',
                                     'QA_PIXEL_mean_7', 'QA_RADSAT_mean_7', 'QA_PIXEL_mean_8', 'QA_RADSAT_mean_8', 'QA_PIXEL_mean_9', 'QA_RADSAT_mean_9',
                                     'class_x', 'class_y', '.geo_x', '.geo_y', '.geo', 'system:index'], 1)
display(df_merged_removeQA)










data_frames = [db_MarchApril2018_df, db_MayJune2018_df, db_JulyAug2018_df,
               db_MarchApril2019_df, db_MayJune2019_df, db_JulyAug2019_df,
               db_MarchApril2020_df, db_MayJune2020_df, db_JulyAug2020_df]

from functools import reduce
df_merged = reduce(lambda left,right: pd.merge(left, right, on='system:index', how='outer'), data_frames).fillna(np.nan)
#display(df_merged)

df_merged_dropna = df_merged.dropna(axis=0, how = 'any')
#display(df_merged_dropna)


df_merged_removeQA = df_merged_dropna.drop(['QA_PIXEL_mean_1', 'QA_RADSAT_mean_1', 'QA_PIXEL_mean_2', 'QA_RADSAT_mean_2', 'QA_PIXEL_mean_3', 'QA_RADSAT_mean_3',
                                     'QA_PIXEL_mean_4', 'QA_RADSAT_mean_4', 'QA_PIXEL_mean_5', 'QA_RADSAT_mean_5', 'QA_PIXEL_mean_6', 'QA_RADSAT_mean_6',
                                     'QA_PIXEL_mean_7', 'QA_RADSAT_mean_7', 'QA_PIXEL_mean_8', 'QA_RADSAT_mean_8', 'QA_PIXEL_mean_9', 'QA_RADSAT_mean_9',
                                     'class_x', 'class_y', '.geo_x', '.geo_y', '.geo', 'system:index'], 1)
display(df_merged_removeQA)



# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

train_db_dict = fc_to_dict(subset_train_db).getInfo()
train_df = pd.DataFrame(train_db_dict)
display(train_df)
#print(nbr_df.dtypes)




# Get Landsat-8 Imagery from 2018, 2019, and 2020 (Centered on 2019 - corresponding to NLCD 2019 land cover classes in the training points)

In [ ]:

##########
## 2018 ##
##########

etmColMarchApril2018 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2018-03-01', '2018-04-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean')

oliColMarchApril2018 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2018-03-01', '2018-04-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

MarchApril2018 = ee.ImageCollection([etmColMarchApril2018, oliColMarchApril2018])

etmColMarchApril2018 = MarchApril2018.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_1(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_1', 'Green_mean_1', 'Red_mean_1', 'NIR_mean_1','SWIR1_mean_1', 'SWIR2_mean_1', 'QA_PIXEL_mean_1', 'QA_RADSAT_mean_1', 'NDVI_mean_1'])

etmColMarchApril2018 = renameEtm_mean_1(etmColMarchApril2018)

etmColMayJune2018 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2018-05-01', '2018-06-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean')

oliColMayJune2018 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2018-05-01', '2018-06-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

MayJune2018 = ee.ImageCollection([etmColMayJune2018, oliColMayJune2018])

etmColMayJune2018 = MayJune2018.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_2(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_2', 'Green_mean_2', 'Red_mean_2', 'NIR_mean_2','SWIR1_mean_2', 'SWIR2_mean_2', 'QA_PIXEL_mean_2', 'QA_RADSAT_mean_2', 'NDVI_mean_2'])

etmColMayJune2018 = renameEtm_mean_2(etmColMayJune2018)

etmColJulyAug2018 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2018-07-01', '2018-08-31') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean') \

oliColJulyAug2018 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2018-07-01', '2018-08-31') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

JulyAug2018 = ee.ImageCollection([etmColJulyAug2018, oliColJulyAug2018])

etmColJulyAug2018 = JulyAug2018.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_3(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_3', 'Green_mean_3', 'Red_mean_3', 'NIR_mean_3','SWIR1_mean_3', 'SWIR2_mean_3', 'QA_PIXEL_mean_3', 'QA_RADSAT_mean_3', 'NDVI_mean_3'])

etmColJulyAug2018 = renameEtm_mean_3(etmColJulyAug2018)





##########
## 2019 ##
##########

etmColMarchApril2019 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2019-03-01', '2019-04-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean')

oliColMarchApril2019 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2019-03-01', '2019-04-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

MarchApril2019 = ee.ImageCollection([etmColMarchApril2019, oliColMarchApril2019])

etmColMarchApril2019 = MarchApril2019.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_4(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_4', 'Green_mean_4', 'Red_mean_4', 'NIR_mean_4','SWIR1_mean_4', 'SWIR2_mean_4', 'QA_PIXEL_mean_4', 'QA_RADSAT_mean_4', 'NDVI_mean_4'])

etmColMarchApril2019 = renameEtm_mean_4(etmColMarchApril2019)

etmColMayJune2019 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2019-05-01', '2019-06-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean')

oliColMayJune2019 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2019-05-01', '2019-06-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

MayJune2019 = ee.ImageCollection([etmColMayJune2019, oliColMayJune2019])

etmColMayJune2019 = MayJune2019.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_5(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_5', 'Green_mean_5', 'Red_mean_5', 'NIR_mean_5','SWIR1_mean_5', 'SWIR2_mean_5', 'QA_PIXEL_mean_5', 'QA_RADSAT_mean_5', 'NDVI_mean_5'])

etmColMayJune2019 = renameEtm_mean_5(etmColMayJune2019)

etmColJulyAug2019 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2019-07-01', '2019-08-31') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean') \

oliColJulyAug2019 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2019-07-01', '2019-08-31') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

JulyAug2019 = ee.ImageCollection([etmColJulyAug2019, oliColJulyAug2019])

etmColJulyAug2019 = JulyAug2019.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_6(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_6', 'Green_mean_6', 'Red_mean_6', 'NIR_mean_6','SWIR1_mean_6', 'SWIR2_mean_6', 'QA_PIXEL_mean_6', 'QA_RADSAT_mean_6', 'NDVI_mean_6'])

etmColJulyAug2019 = renameEtm_mean_6(etmColJulyAug2019)


##########
## 2020 ##
##########

etmColMarchApril2020 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2020-03-01', '2020-04-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean')

oliColMarchApril2020 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2020-03-01', '2020-04-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

MarchApril2020 = ee.ImageCollection([etmColMarchApril2020, oliColMarchApril2020])

etmColMarchApril2020 = MarchApril2020.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_7(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_7', 'Green_mean_7', 'Red_mean_7', 'NIR_mean_7','SWIR1_mean_7', 'SWIR2_mean_7', 'QA_PIXEL_mean_7', 'QA_RADSAT_mean_7', 'NDVI_mean_7'])

etmColMarchApril2020 = renameEtm_mean_7(etmColMarchApril2020)

etmColMayJune2020 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2020-05-01', '2020-06-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean')

oliColMayJune2020 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2020-05-01', '2020-06-30') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

MayJune2020 = ee.ImageCollection([etmColMayJune2020, oliColMayJune2020])

etmColMayJune2020 = MayJune2020.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_8(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_8', 'Green_mean_8', 'Red_mean_8', 'NIR_mean_8','SWIR1_mean_8', 'SWIR2_mean_8', 'QA_PIXEL_mean_8', 'QA_RADSAT_mean_8', 'NDVI_mean_8'])

etmColMayJune2020 = renameEtm_mean_8(etmColMayJune2020)

etmColJulyAug2020 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
  .filterDate('2020-07-01', '2020-08-31') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepEtm) \
  .map(addNDVI) \
  .reduce('mean') \

oliColJulyAug2020 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2020-07-01', '2020-08-31') \
  .filterBounds(pts) \
  .map(maskQuality) \
  .map(prepOli) \
  .map(addNDVI) \
  .reduce('mean')

JulyAug2020 = ee.ImageCollection([etmColJulyAug2020, oliColJulyAug2020])

etmColJulyAug2020 = JulyAug2020.reduce('mean')

# Selects and renames bands of interest for TM/ETM+.
def renameEtm_mean_9(img):
  return img.select(
      ['Blue_mean_mean', 'Green_mean_mean', 'Red_mean_mean', 'NIR_mean_mean', 'SWIR1_mean_mean', 'SWIR2_mean_mean', 'QA_PIXEL_mean_mean', 'QA_RADSAT_mean_mean', 'NDVI_mean_mean'],
      ['Blue_mean_9', 'Green_mean_9', 'Red_mean_9', 'NIR_mean_9','SWIR1_mean_9', 'SWIR2_mean_9', 'QA_PIXEL_mean_9', 'QA_RADSAT_mean_9', 'NDVI_mean_9'])

etmColJulyAug2020 = renameEtm_mean_9(etmColJulyAug2020)

etmColJulyAug2020.getInfo()


#Extract Landsat-8 Bands from Points, Format into Pandas DataFrame

In [ ]:

# Sample Regions

# 2018
etmColMarchApril2018_fc = etmColMarchApril2018.sampleRegions(collection=pts, properties=['class'], scale=30)
#print(etmColMarchApril2018_fc.getInfo())
etmColMayJune2018_fc = etmColMayJune2018.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColMayJune2018_fc.getInfo())
etmColJulyAug2018_fc = etmColJulyAug2018.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColJulyAug2018_fc.getInfo())

# 2019
etmColMarchApril2019_fc = etmColMarchApril2019.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColMarchApril2019_fc.getInfo())
etmColMayJune2019_fc = etmColMayJune2019.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColMayJune2019_fc.getInfo())
etmColJulyAug2019_fc = etmColJulyAug2019.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColJulyAug2019_fc.getInfo())

# 2020
etmColMarchApril2020_fc = etmColMarchApril2020.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColMarchApril2020_fc.getInfo())
etmColMayJune2020_fc = etmColMayJune2020.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColMayJune2020_fc.getInfo())
etmColJulyAug2020_fc = etmColJulyAug2020.sampleRegions(collection=pts, properties=['class'], scale=30, geometries=True)
#print(etmColJulyAug2020_fc.getInfo())



# Export feature class assets
outputBucket = 'landcover_samples_nlcd2019_onemillionpoints'
# Make sure the bucket exists.
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + outputBucket) 
    else 'Output Cloud Storage bucket does not exist.')


# 2018
task1 = ee.batch.Export.table.toCloudStorage(
    collection=etmColMarchApril2018_fc,
    description='etmColMarchApril2018_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColMarchApril2018_fc_landcover',
    fileFormat='CSV')

task2 = ee.batch.Export.table.toCloudStorage(
    collection=etmColMayJune2018_fc,
    description='etmColMayJune2018_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColMayJune2018_fc_landcover',
    fileFormat='CSV')

task3 = ee.batch.Export.table.toCloudStorage(
    collection=etmColJulyAug2018_fc,
    description='etmColJulyAug2018_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColJulyAug2018_fc_landcover',
    fileFormat='CSV')

# 2019
task4 = ee.batch.Export.table.toCloudStorage(
    collection=etmColMarchApril2019_fc,
    description='etmColMarchApril2019_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColMarchApril2019_fc_landcover',
    fileFormat='CSV')

task5 = ee.batch.Export.table.toCloudStorage(
    collection=etmColMayJune2019_fc,
    description='etmColMayJune2019_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColMayJune2019_fc_landcover',
    fileFormat='CSV')

task6 = ee.batch.Export.table.toCloudStorage(
    collection=etmColJulyAug2019_fc,
    description='etmColJulyAug2019_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColJulyAug2019_fc_landcover',
    fileFormat='CSV')

# 2020
task7 = ee.batch.Export.table.toCloudStorage(
    collection=etmColMarchApril2020_fc,
    description='etmColMarchApril2020_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColMarchApril2020_fc_landcover',
    fileFormat='CSV')

task8 = ee.batch.Export.table.toCloudStorage(
    collection=etmColMayJune2020_fc,
    description='etmColMayJune2020_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColMayJune2020_fc_landcover',
    fileFormat='CSV')

task9 = ee.batch.Export.table.toCloudStorage(
    collection=etmColJulyAug2020_fc,
    description='etmColJulyAug2020_fc_landcover export',
    bucket = outputBucket,
    fileNamePrefix='etmColJulyAug2020_fc_landcover',
    fileFormat='CSV')

#Export/Start Tasks

task1.start()
task2.start()
task3.start()
task4.start()
task5.start()
task6.start()
task7.start()
task8.start()
task9.start()




In [ ]:

# Re-Import Feature Class Assets (no longer hosted on EE and thus not restricted to size limits) and format to pandas DataFrame
# Depends on import gcsfs

#2018
db_MarchApril2018_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColMarchApril2018_fc_landcover.csv')
#display(db_MarchApril2018_df)
db_MayJune2018_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColMayJune2018_fc_landcover.csv')
#display(db_MayJune2018_df)
db_JulyAug2018_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColJulyAug2018_fc_landcover.csv')
#display(db_JulyAug2018_df)


#2019
db_MarchApril2019_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColMarchApril2019_fc_landcover.csv')
#display(db_MarchApril2019_df)
db_MayJune2019_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColMayJune2019_fc_landcover.csv')
#display(db_MayJune2019_df)
db_JulyAug2019_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColJulyAug2019_fc_landcover.csv')
#display(db_JulyAug2019_df)


#2020
db_MarchApril2020_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColMarchApril2020_fc_landcover.csv')
#display(db_MarchApril2020_df)
db_MayJune2020_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColMayJune2020_fc_landcover.csv')
#display(db_MayJune2020_df)
db_JulyAug2020_df = pd.read_csv('gs://landcover_samples_nlcd2019_onemillionpoints/etmColJulyAug2020_fc_landcover.csv')
#display(db_JulyAug2020_df)

#Merge Landsat-8 <> LandCover Points into one DataFrame

In [ ]:

data_frames = [db_MarchApril2018_df, db_MayJune2018_df, db_JulyAug2018_df,
               db_MarchApril2019_df, db_MayJune2019_df, db_JulyAug2019_df,
               db_MarchApril2020_df, db_MayJune2020_df, db_JulyAug2020_df]

from functools import reduce
df_merged = reduce(lambda left,right: pd.merge(left, right, on='system:index', how='outer'), data_frames).fillna(np.nan)
#display(df_merged)

df_merged_dropna = df_merged.dropna(axis=0, how = 'any')
#display(df_merged_dropna)


df_merged_removeQA = df_merged_dropna.drop(['QA_PIXEL_mean_1', 'QA_RADSAT_mean_1', 'QA_PIXEL_mean_2', 'QA_RADSAT_mean_2', 'QA_PIXEL_mean_3', 'QA_RADSAT_mean_3',
                                     'QA_PIXEL_mean_4', 'QA_RADSAT_mean_4', 'QA_PIXEL_mean_5', 'QA_RADSAT_mean_5', 'QA_PIXEL_mean_6', 'QA_RADSAT_mean_6',
                                     'QA_PIXEL_mean_7', 'QA_RADSAT_mean_7', 'QA_PIXEL_mean_8', 'QA_RADSAT_mean_8', 'QA_PIXEL_mean_9', 'QA_RADSAT_mean_9',
                                     'class_x', 'class_y', '.geo_x', '.geo_y', '.geo', 'system:index'], 1)
display(df_merged_removeQA)



#Export DataFrame to CSV

In [ ]:
df_merged_removeQA.to_csv("example2_1dcnn_april2022.csv")

#Functions to read and compute spectral features on SITS 


In [ ]:

""" 
	Some functions to read and compute spectral features on SITS
"""


import sys, os
import numpy as np
import pandas as pd
import math
import random
import itertools

import csv

#-----------------------------------------------------------------------
#---------------------- SATELLITE MODULE
#-----------------------------------------------------------------------
#final_class_label = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12']


#-----------------------------------------------------------------------
def readSITSData(name_file):
	"""
		Read the data contained in name_file
		INPUT:
			- name_file: file where to read the data
		OUTPUT:
			- X: variable vectors for each example
			- polygon_ids: id polygon (use e.g. for validation set)
			- Y: label for each example
	"""
	
	data = pd.read_table(name_file, sep=',', header=None)
	
	y_data = data.iloc[:,0]
	y = np.asarray(y_data.values, dtype='uint8')
	
	polygonID_data = data.iloc[:,1]
	polygon_ids = polygonID_data.values
	polygon_ids = np.asarray(polygon_ids, dtype='uint16')
		
	X_data = data.iloc[:,2:]
	X = X_data.values
	X = np.asarray(X, dtype='float32')

	return  X, polygon_ids, y


#-----------------------------------------------------------------------
def addFeatures(X):
	"""
		Read the data contained in name_file
		INPUT:
			- X: orginal X features composed of threes bands (NIR-R-G)
				in the following order 
					[date1.NIR, date1.R, date1.G, ..., dateD.NIR, dateD.R, dateD.G]
		OUTPUT:
			- X_features: orginal_X with the addition of NDVI, NDWI and Brilliance
				in the following order	
					[X, date1.NDVI, ..., dateD.NDVI, date1.NDWI, ..., dateD.NDWI, date1.Brilliance, ..., dateD.Brilliance]
	"""
	n_channels = 3
	
	NIR = X[:,0::n_channels]
	NIR = np.array(NIR)
	NIR = NIR.astype(np.float)
	R = X[:,1::n_channels]
	R = np.array(R)
	R = R.astype(np.float)
	G = X[:,2::n_channels]
	G = np.array(G)
	G = G.astype(np.float)	
	
	NDVI = np.where(NIR+R!=0., (NIR-R)/(NIR+R), 0.)
	NDVI = NDVI.astype(float)
	
	
	NDWI = np.where(G+NIR!=0., (G-NIR)/(G+NIR), 0.)
	NDWI = NDWI.astype(float)
	
	Brilliance = np.sqrt((NIR*NIR + R*R + G*G)/3.0)
	Brilliance = Brilliance.astype(float)
	
	return NDVI, NDWI, Brilliance

#-----------------------------------------------------------------------
def computeNDVI(X, n_channels):
	"""
		Read the data contained in name_file
		INPUT:
			- X: orginal X features composed of threes bands (NIR-R-G)
				in the following order 
					[date1.NIR, date1.R, date1.G, ..., dateD.NIR, dateD.R, dateD.G]
		OUTPUT:
			- X_features: orginal_X with the addition of NDVI, NDWI and Brilliance
				in the following order	
					[X, date1.NDVI, ..., dateD.NDVI, date1.NDWI, ..., dateD.NDWI, date1.Brilliance, ..., dateD.Brilliance]
	"""
	
	NIR = X[:,0::n_channels]
	NIR = np.array(NIR)
	NIR = NIR.astype(np.float)
	R = X[:,1::n_channels]
	R = np.array(R)
	R = R.astype(np.float)
	
	NDVI = np.where(NIR+R!=0., (NIR-R)/(NIR+R), 0.)
	return NDVI

#-----------------------------------------------------------------------
def addingfeat_reshape_data(X, feature_strategy, nchannels):
	"""
		Reshaping (feature format (3 bands): d1.b1 d1.b2 d1.b3 d2.b1 d2.b2 d2.b3 ...)
		INPUT:
			-X: original feature vector ()
			-feature_strategy: used features (options: SB, NDVI, SB3feat)
			-nchannels: number of channels
		OUTPUT:
			-new_X: data in the good format for Keras models
	"""
			
	if feature_strategy=='SB':
		print("SPECTRAL BANDS-----------------------------------------")
		return X.reshape(X.shape[0],int(X.shape[1]/nchannels),nchannels)
								
	elif feature_strategy=='NDVI':
		print("NDVI only----------------------------------------------")
		new_X = computeNDVI(X, nchannels)
		return np.expand_dims(new_X, axis=2)
							
	elif feature_strategy=='SB3feat':
		print("SB + NDVI + NDWI + IB----------------------------------")
		NDVI, NDWI, IB = addFeatures(X)		
		new_X = X.reshape(X.shape[0],int(X.shape[1]/nchannels),nchannels)		
		new_X = np.dstack((new_X, NDVI))
		new_X = np.dstack((new_X, NDWI))
		new_X = np.dstack((new_X, IB))
		return new_X
	else:
		print("Not referenced!!!-------------------------------------------")
		return -1

#-----------------------------------------------------------------------
def computingMinMax(X, per=2):
	min_per = np.percentile(X, per, axis=(0,1))
	max_per = np.percentile(X, 100-per, axis=(0,1))
	return min_per, max_per

#-----------------------------------------------------------------------
def read_minMaxVal(minmax_file):	
	with open(minmax_file, 'r') as f:
		reader = csv.reader(f, delimiter=',')
		min_per = next(reader)
		max_per = next(reader)
	min_per = [float(k) for k in min_per]
	min_per = np.array(min_per)
	max_per = [float(k) for k in max_per]
	max_per = np.array(max_per)
	return min_per, max_per

#-----------------------------------------------------------------------
def save_minMaxVal(minmax_file, min_per, max_per):	
	with open(minmax_file, 'w') as f:
		writer = csv.writer(f, delimiter=',')
		writer.writerow(min_per)
		writer.writerow(max_per)

#-----------------------------------------------------------------------
def normalizingData(X, min_per, max_per):
	return (X-min_per)/(max_per-min_per)

#-----------------------------------------------------------------------	
def extractValSet(X_train, polygon_ids_train, y_train, val_rate=0.1):
	unique_pol_ids_train, indices = np.unique(polygon_ids_train, return_inverse=True) #-- pold_ids_train = unique_pol_ids_train[indices]
	nb_pols = len(unique_pol_ids_train)
	
	ind_shuffle = list(range(nb_pols))
	random.shuffle(ind_shuffle)
	list_indices = [[] for i in range(nb_pols)]
	shuffle_indices = [[] for i in range(nb_pols)]
	[ list_indices[ind_shuffle[val]].append(idx) for idx, val in enumerate(indices)]					
		
	final_ind = list(itertools.chain.from_iterable(list_indices))
	m = len(final_ind)
	final_train = int(math.ceil(m*(1.0-val_rate)))
	
	shuffle_polygon_ids_train = polygon_ids_train[final_ind]
	id_final_train = shuffle_polygon_ids_train[final_train]
	
	while shuffle_polygon_ids_train[final_train-1]==id_final_train:
		final_train = final_train-1
	
	
	new_X_train = X_train[final_ind[:final_train],:,:]
	new_y_train = y_train[final_ind[:final_train]]
	new_X_val = X_train[final_ind[final_train:],:,:]
	new_y_val = y_train[final_ind[final_train:]]
	
	return new_X_train, new_y_train, new_X_val, new_y_val
	

In [ ]:
from tensorflow.keras.utils import to_categorical

open("/content/drive/My Drive/Invasives Research UMN/Remote Sensing Master/Leafy Spurge Demography/temporalCNN-master/example/train_dataset.csv").read()

res_path = '/content/drive/My Drive/Invasives Research UMN/Remote Sensing Master/Leafy Spurge Demography'
sits_path = '/content/drive/My Drive/Invasives Research UMN/Remote Sensing Master/Leafy Spurge Demography/temporalCNN-master/example'
feature = "SB"
noarchi = 2
norun = 0


#-- Creating output path if does not exist
if not os.path.exists(res_path):
  print("ResPath DNE")
  os.makedirs(res_path)
	
	#---- Parameters to set
n_channels = 7 #-- B G NDVI NIR Red SWIR1 SWIR2
val_rate = 0.00

	#---- Evaluated metrics
eval_label = ['OA', 'train_loss', 'train_time', 'test_time']	
	
	#---- String variables
train_str = 'train_dataset'
test_str = 'test_dataset'					
	#---- Get filenames
train_file = sits_path + '/' + train_str + '.csv'
test_file = sits_path + '/' + test_str + '.csv'
print("train_file: ", train_file)
print("test_file: ", test_file)
	
	#---- output files			
res_path = res_path + '/Archi' + str(noarchi) + '/'
if not os.path.exists(res_path):
  os.makedirs(res_path)
  print("noarchi: ", noarchi)
	
str_result = feature + '-' + train_str + '-noarchi' + str(noarchi) + '-norun' + str(norun) 
res_file = res_path + '/resultOA-' + str_result + '.csv'
res_mat = np.zeros((len(eval_label),1))
traintest_loss_file = res_path + '/trainingHistory-' + str_result + '.csv'
conf_file = res_path + '/confMatrix-' + str_result + '.csv'
out_model_file = res_path + '/bestmodel-' + str_result + '.h5'


	#---- Downloading
X_train, polygon_ids_train, y_train = readSITSData(train_file)

print(X_train.shape) #13336, 63
X_test,  polygon_ids_test, y_test = readSITSData(test_file)
print(X_train)
print(polygon_ids_train)
print(y_train.shape) #13336

n_classes_test = len(np.unique(y_test))
print(n_classes_test)
n_classes_train = len(np.unique(y_train))
print(n_classes_train)
if(n_classes_test != n_classes_train):
  print("WARNING: different number of classes in train and test")
n_classes = max(n_classes_train, n_classes_test)
y_train_one_hot = to_categorical(y_train) #specify number of classes explicity - may need to recode classes sequentially (1-9) to work correctly
y_test_one_hot = to_categorical(y_test)

print(y_train_one_hot)
print(y_test_one_hot)
	
	#---- Adding the features and reshaping the data if necessary
X_train = addingfeat_reshape_data(X_train, feature, n_channels) #Feature = "SB" (spectral bands)

print(X_train[0, :, :])
print(X_train.shape)
X_test = addingfeat_reshape_data(X_test, feature, n_channels)		
print(X_test.shape)

#---- Normalizing the data per band (Do we want to normalize across years or within one year?)
minMaxVal_file = '.'.join(out_model_file.split('.')[0:-1])
minMaxVal_file = minMaxVal_file + '_minMax.txt'

if not os.path.exists(minMaxVal_file): 
  min_per, max_per = computingMinMax(X_train) #compute 98% min/max (per = 2) on bands
  save_minMaxVal(minMaxVal_file, min_per, max_per)
else:
  min_per, max_per = read_minMaxVal(minMaxVal_file)

print(min_per, max_per)

X_train =  normalizingData(X_train, min_per, max_per)
X_test =  normalizingData(X_test, min_per, max_per)

print(X_train) #verify normalization worked as intended






#Define Keras Model Architectures

https://github.com/charlotte-pel/temporalCNN/


In [ ]:

""" 
	Defining keras architecre, and training the models
"""

import sys, os
import numpy as np
import time

import keras
from keras import layers
from keras import optimizers
from keras.regularizers import l2
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Flatten, Lambda, SpatialDropout1D, Concatenate
from keras.layers import Conv1D, Conv2D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.callbacks import Callback, ModelCheckpoint, History, EarlyStopping
from keras.models import Model, load_model
from keras.optimizers import *
from keras.utils.np_utils import to_categorical
from keras import backend as K



#-----------------------------------------------------------------------
#---------------------- Modules
#-----------------------------------------------------------------------

#-----------------------------------------------------------------------		
def conv_bn(X, **conv_params):	
	nbunits = conv_params["nbunits"];
	kernel_size = conv_params["kernel_size"];

	strides = conv_params.setdefault("strides", 1)
	padding = conv_params.setdefault("padding", "same")
	kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-6))
	kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")

	Z = Conv1D(nbunits, kernel_size=kernel_size, 
			strides = strides, padding=padding,
			kernel_initializer=kernel_initializer,
			kernel_regularizer=kernel_regularizer)(X)

	return BatchNormalization(axis=-1)(Z) #-- CHANNEL_AXIS (-1)

#-----------------------------------------------------------------------		
def conv_bn_relu(X, **conv_params):
	Znorm = conv_bn(X, **conv_params)
	return Activation('relu')(Znorm)
	
#-----------------------------------------------------------------------		
def conv_bn_relu_drop(X, **conv_params):	
	dropout_rate = conv_params.setdefault("dropout_rate", 0.5)
	A = conv_bn_relu(X, **conv_params)
	return Dropout(dropout_rate)(A)

#-----------------------------------------------------------------------		
def conv_bn_relu_spadrop(X, **conv_params):	
	dropout_rate = conv_params.setdefault("dropout_rate", 0.5)
	A = conv_bn_relu(X, **conv_params)
	return SpatialDropout1D(dropout_rate)(A)

#-----------------------------------------------------------------------		
def conv2d_bn(X, **conv_params):	
	nbunits = conv_params["nbunits"];
	kernel_size = conv_params["kernel_size"];

	strides = conv_params.setdefault("strides", 1)
	padding = conv_params.setdefault("padding", "same")
	kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-6))
	kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")

	Z = Conv2D(nbunits, kernel_size=kernel_size, 
			strides = strides, padding=padding,
			kernel_initializer=kernel_initializer,
			kernel_regularizer=kernel_regularizer)(X)

	return BatchNormalization(axis=-1)(Z) #-- CHANNEL_AXIS (-1)

#-----------------------------------------------------------------------		
def conv2d_bn_relu(X, **conv_params):
	Znorm = conv2d_bn(X, **conv_params)
	return Activation('relu')(Znorm)
	
#-----------------------------------------------------------------------		
def conv2d_bn_relu_drop(X, **conv_params):	
	dropout_rate = conv_params.setdefault("dropout_rate", 0.5)
	A = conv2d_bn_relu(X, **conv_params)
	return Dropout(dropout_rate)(A)

#-----------------------------------------------------------------------		
def conv2d_bn_relu_spadrop(X, **conv_params):	
	dropout_rate = conv_params.setdefault("dropout_rate", 0.5)
	A = conv2d_bn_relu(X, **conv_params)
	return SpatialDropout1D(dropout_rate)(A)

	
#-----------------------------------------------------------------------		
def relu_drop(X, **conv_params):	
	dropout_rate = conv_params.setdefault("dropout_rate", 0.5)
	A = Activation('relu')(X)
	return Dropout(dropout_rate)(A)

#-----------------------------------------------------------------------		
def fc_bn(X, **fc_params):
	nbunits = fc_params["nbunits"];
	
	kernel_regularizer = fc_params.setdefault("kernel_regularizer", l2(1.e-6))
	kernel_initializer = fc_params.setdefault("kernel_initializer", "he_normal")
		
	Z = Dense(nbunits, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)(X)
	return BatchNormalization(axis=-1)(Z) #-- CHANNEL_AXIS (-1)
	
#-----------------------------------------------------------------------		
def fc_bn_relu(X, **fc_params):	
	Znorm = fc_bn(X, **fc_params)
	return Activation('relu')(Znorm)

#-----------------------------------------------------------------------		
def fc_bn_relu_drop(X, **fc_params):
	dropout_rate = fc_params.setdefault("dropout_rate", 0.5)
	A = fc_bn_relu(X, **fc_params)
	return Dropout(dropout_rate)(A)

#-----------------------------------------------------------------------		
def softmax(X, nbclasses, **params):
	kernel_regularizer = params.setdefault("kernel_regularizer", l2(1.e-6))
	kernel_initializer = params.setdefault("kernel_initializer", "glorot_uniform")
	return Dense(nbclasses, activation='softmax', 
			kernel_initializer=kernel_initializer,
			kernel_regularizer=kernel_regularizer)(X)

#-----------------------------------------------------------------------		
def getNoClasses(model_path):
	model = load_model(model_path)
	last_weight = model.get_weights()[-1]
	nclasses = last_weight.shape[0] #--- get size of the bias in the Softmax
	return nclasses

#-----------------------------------------------------------------------
#---------------------- Training models
#-----------------------------------------------------------------------
#-----------------------------------------------------------------------
def trainTestModel(model, X_train, Y_train_onehot, X_test, Y_test_onehot, out_model_file, **train_params):
	#---- variables
	n_epochs = train_params.setdefault("n_epochs", 20)
	batch_size = train_params.setdefault("batch_size", 32)
	
	lr = train_params.setdefault("lr", 0.001)
	beta_1 = train_params.setdefault("beta_1", 0.9)
	beta_2 = train_params.setdefault("beta_2", 0.999)
	decay = train_params.setdefault("decay", 0.0)

	#---- optimizer
	opt = tf.keras.optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, decay=decay)
	model.compile(optimizer = opt, loss = "categorical_crossentropy",
			metrics = ["accuracy"])
	
	#---- monitoring the minimum loss
	checkpoint = ModelCheckpoint(out_model_file, monitor='loss',
			verbose=0, save_best_only=True, mode='min')
	callback_list = [checkpoint]
		
	start_train_time = time.time()
	hist = model.fit(x = X_train, y = Y_train_onehot, epochs = n_epochs, 
		batch_size = batch_size, shuffle=True,
		validation_data=(X_test, Y_test_onehot),
		verbose=1, callbacks=callback_list)
	train_time = round(time.time()-start_train_time, 2)
		
	#-- download the best model
	del model	
	model = load_model(out_model_file)
	start_test_time = time.time()
	test_loss, test_acc = model.evaluate(x=X_test, y=Y_test_onehot, 
		batch_size = 128, verbose=0)
	test_time = round(time.time()-start_test_time, 2)
	
	return test_acc, np.min(hist.history['loss']), model, hist.history, train_time, test_time

#-----------------------------------------------------------------------
def trainTestModel_EarlyAbandon(model, X_train, Y_train_onehot, X_test, Y_test_onehot, out_model_file, **train_params):
	#---- variables
	n_epochs = train_params.setdefault("n_epochs", 20)
	batch_size = train_params.setdefault("batch_size", 32)
	
	lr = train_params.setdefault("lr", 0.001)
	beta_1 = train_params.setdefault("beta_1", 0.9)
	beta_2 = train_params.setdefault("beta_2", 0.999)
	decay = train_params.setdefault("decay", 0.0)

	#---- optimizer
	opt = tf.keras.optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, decay=decay)
	model.compile(optimizer = opt, loss = "categorical_crossentropy",
			metrics = ["accuracy"])
	
	#---- monitoring the minimum loss
	checkpoint = ModelCheckpoint(out_model_file, monitor='loss',
			verbose=0, save_best_only=True, mode='min')
	#early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=0, verbose=0, mode='auto')
  #callback_list = [checkpoint, early_stop]
	callback_list = [checkpoint]
			
	start_train_time = time.time()
	hist = model.fit(x = X_train, y = Y_train_onehot, epochs = n_epochs, 
		batch_size = batch_size, shuffle=True,
		validation_data=(X_test, Y_test_onehot),
		verbose=1, callbacks=callback_list)
	train_time = round(time.time()-start_train_time, 2)
		
	#-- download the best model
	del model	
	model = load_model(out_model_file)
	start_test_time = time.time()
	test_loss, test_acc = model.evaluate(x=X_test, y=Y_test_onehot, 
		batch_size = 128, verbose=0)
	test_time = round(time.time()-start_test_time, 2)
	
	return test_acc, np.min(hist.history['loss']), model, hist.history, train_time, test_time
		
#-----------------------------------------------------------------------
def trainValTestModel(model, X_train, Y_train_onehot, X_val, Y_val_onehot, X_test, Y_test_onehot, out_model_file, **train_params):
	#---- variables
	n_epochs = train_params.setdefault("n_epochs", 20)
	batch_size = train_params.setdefault("batch_size", 32)
	
	lr = train_params.setdefault("lr", 0.001)
	beta_1 = train_params.setdefault("beta_1", 0.9)
	beta_2 = train_params.setdefault("beta_2", 0.999)
	decay = train_params.setdefault("decay", 0.0)

	#---- optimizer
	opt = tf.keras.optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, decay=decay)
	model.compile(optimizer = opt, loss = "categorical_crossentropy",
			metrics = ["accuracy"])
	
	#---- monitoring the minimum validation loss
	checkpoint = ModelCheckpoint(out_model_file, monitor='val_loss',
			verbose=0, save_best_only=True, mode='min')
	callback_list = [checkpoint]
		
	start_train_time = time.time()
	hist = model.fit(x = X_train, y = Y_train_onehot, epochs = n_epochs, 
		batch_size = batch_size, shuffle=True,
		validation_data=(X_val, Y_val_onehot),
		verbose=1, callbacks=callback_list)
	train_time = round(time.time()-start_train_time, 2)
		
	#-- download the best model
	del model	
	model = load_model(out_model_file)
	start_test_time = time.time()
	test_loss, test_acc = model.evaluate(x=X_test, y=Y_test_onehot, 
		batch_size = 128, verbose=0)
	test_time = round(time.time()-start_test_time, 2)
	
	return test_acc, np.min(hist.history['val_loss']), model, hist.history, train_time, test_time
	
#-----------------------------------------------------------------------
def trainValTestModel_EarlyAbandon(model, X_train, Y_train_onehot, X_val, Y_val_onehot, X_test, Y_test_onehot, out_model_file, **train_params):
	#---- variables
	n_epochs = train_params.setdefault("n_epochs", 20)
	batch_size = train_params.setdefault("batch_size", 32)
	
	lr = train_params.setdefault("lr", 0.001)
	beta_1 = train_params.setdefault("beta_1", 0.9)
	beta_2 = train_params.setdefault("beta_2", 0.999)
	decay = train_params.setdefault("decay", 0.0)

	#---- optimizer
	opt = tf.keras.optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, decay=decay)
	model.compile(optimizer = opt, loss = "categorical_crossentropy",
			metrics = ["accuracy"])
	
	#---- monitoring the minimum validation loss
	checkpoint = ModelCheckpoint(out_model_file, monitor='val_loss',
			verbose=0, save_best_only=True, mode='min')
	early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
	callback_list = [checkpoint, early_stop]
		
	start_train_time = time.time()
	hist = model.fit(x = X_train, y = Y_train_onehot, epochs = n_epochs, 
		batch_size = batch_size, shuffle=True,
		validation_data=(X_val, Y_val_onehot),
		verbose=1, callbacks=callback_list)
	train_time = round(time.time()-start_train_time, 2)
		
	#-- download the best model
	del model	
	model = load_model(out_model_file)
	start_test_time = time.time()
	test_loss, test_acc = model.evaluate(x=X_test, y=Y_test_onehot, 
		batch_size = 128, verbose=0)
	test_time = round(time.time()-start_test_time, 2)
	
	return test_acc, np.min(hist.history['val_loss']), model, hist.history, train_time, test_time


In [ ]:

""" 
	Defining keras architecture.
	4.4. How big and deep model for our data?
	4.4.1. Width influence or the bias-variance trade-off
"""

import sys, os

import keras
from keras import layers
from keras.layers import Flatten
from keras import backend as K

#-----------------------------------------------------------------------
#---------------------- ARCHITECTURES
#------------------------------------------------------------------------	

#-----------------------------------------------------------------------		
def Archi_3CONV16_1FC256(X, nbclasses):
	
	#-- get the input sizes
	m, L, depth = X.shape
	input_shape = (L,depth)
	
	#-- parameters of the architecture
	l2_rate = 1.e-6
	dropout_rate = 0.5
	nb_conv = 3
	nb_fc= 1
	nbunits_conv = 16 #-- will be double
	nbunits_fc = 256 #-- will be double
	
	# Define the input placeholder.
	X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
	X = X_input
	for add in range(nb_conv):
		X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
	X = Flatten()(X)
	for add in range(nb_fc):	
		X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
	out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
	return Model(inputs = X_input, outputs = out, name='Archi_3CONV16_1FC256')	
	
	
#-----------------------------------------------------------------------		
def Archi_3CONV32_1FC256(X, nbclasses):
	
	#-- get the input sizes
	m, L, depth = X.shape
	input_shape = (L,depth)
	
	#-- parameters of the architecture
	l2_rate = 1.e-6
	dropout_rate = 0.5
	nb_conv = 3
	nb_fc= 1
	nbunits_conv = 32 #-- will be double
	nbunits_fc = 256 #-- will be double
	
	# Define the input placeholder.
	X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
	X = X_input
	for add in range(nb_conv):
		X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
	X = Flatten()(X)
	for add in range(nb_fc):	
		X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
	out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
	return Model(inputs = X_input, outputs = out, name='Archi_3CONV32_1FC256')	


#-----------------------------------------------------------------------		
def Archi_3CONV64_1FC256(X, nbclasses):
	
	#-- get the input sizes
	m, L, depth = X.shape
	input_shape = (L,depth)
	
	#-- parameters of the architecture
	l2_rate = 1.e-6
	dropout_rate = 0.5
	nb_conv = 3
	nb_fc= 1
	nbunits_conv = 64 #-- will be double
	nbunits_fc = 256 #-- will be double
	
	# Define the input placeholder.
	X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
	X = X_input
	for add in range(nb_conv):
		X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
	X = Flatten()(X)
	for add in range(nb_fc):	
		X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
	out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
	return Model(inputs = X_input, outputs = out, name='Archi_3CONV64_1FC256')	


#-----------------------------------------------------------------------		
def Archi_3CONV128_1FC256(X, nbclasses):
	
	#-- get the input sizes
	m, L, depth = X.shape
	input_shape = (L,depth)
	
	#-- parameters of the architecture
	l2_rate = 1.e-6
	dropout_rate = 0.5
	nb_conv = 3
	nb_fc= 1
	nbunits_conv = 128 #-- will be double
	nbunits_fc = 256 #-- will be double
	
	# Define the input placeholder.
	X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
	X = X_input
	for add in range(nb_conv):
		X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
	X = Flatten()(X)
	for add in range(nb_fc):	
		X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
	out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
	return Model(inputs = X_input, outputs = out, name='Archi_3CONV128_1FC256')	


#-----------------------------------------------------------------------		
def Archi_3CONV256_1FC256(X, nbclasses):
	
	#-- get the input sizes
	m, L, depth = X.shape
	input_shape = (L,depth)
	
	#-- parameters of the architecture
	l2_rate = 1.e-6
	dropout_rate = 0.5
	nb_conv = 3
	nb_fc= 1
	nbunits_conv = 256 #-- will be double
	nbunits_fc = 256 #-- will be double
	
	# Define the input placeholder.
	X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
	X = X_input
	for add in range(nb_conv):
		X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
	X = Flatten()(X)
	for add in range(nb_fc):	
		X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
	out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
	return Model(inputs = X_input, outputs = out, name='Archi_3CONV256_1FC256')	


#-----------------------------------------------------------------------		
def Archi_3CONV512_1FC256(X, nbclasses):
	
	#-- get the input sizes
	m, L, depth = X.shape
	input_shape = (L,depth)
	
	#-- parameters of the architecture
	l2_rate = 1.e-6
	dropout_rate = 0.5
	nb_conv = 3
	nb_fc= 1
	nbunits_conv = 512 #-- will be double
	nbunits_fc = 256 #-- will be double
	
	# Define the input placeholder.
	X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
	X = X_input
	for add in range(nb_conv):
		X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
	X = Flatten()(X)
	for add in range(nb_fc):	
		X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
	out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
	return Model(inputs = X_input, outputs = out, name='Archi_3CONV512_1FC256')	


#-----------------------------------------------------------------------		
def Archi_3CONV1024_1FC256(X, nbclasses):
	
	#-- get the input sizes
	m, L, depth = X.shape
	input_shape = (L,depth)
	
	#-- parameters of the architecture
	l2_rate = 1.e-6
	dropout_rate = 0.5
	nb_conv = 3
	nb_fc= 1
	nbunits_conv = 1024 #-- will be double
	nbunits_fc = 256 #-- will be double
	
	# Define the input placeholder.
	X_input = Input(input_shape)
		
	#-- nb_conv CONV layers
	X = X_input
	for add in range(nb_conv):
		X = conv_bn_relu_drop(X, nbunits=nbunits_conv, kernel_size=5, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
	#-- Flatten + 	1 FC layers
	X = Flatten()(X)
	for add in range(nb_fc):	
		X = fc_bn_relu_drop(X, nbunits=nbunits_fc, kernel_regularizer=l2(l2_rate), dropout_rate=dropout_rate)
		
	#-- SOFTMAX layer
	out = softmax(X, nbclasses, kernel_regularizer=l2(l2_rate))
		
	# Create model.
	return Model(inputs = X_input, outputs = out, name='Archi_3CONV1024_1FC256')	


#--------------------- Switcher for running the architectures
def runArchi(noarchi, *args):
	#---- variables
	n_epochs = 20
	batch_size = 32
	
	switcher = {		
		0: Archi_3CONV16_1FC256,
		1: Archi_3CONV32_1FC256,
		2: Archi_3CONV64_1FC256,
		3: Archi_3CONV128_1FC256,
		3: Archi_3CONV256_1FC256,
		4: Archi_3CONV512_1FC256,
		5: Archi_3CONV1024_1FC256,
	}
	func = switcher.get(noarchi, lambda: 0)
	model = func(args[0], args[1].shape[1])
	
	if len(args)==5:
		return trainTestModel_EarlyAbandon(model, *args, n_epochs=n_epochs, batch_size=batch_size)
	elif len(args)==7:
		return trainValTestModel_EarlyAbandon(model, *args, n_epochs=n_epochs, batch_size=batch_size)


In [ ]:

#---- Extracting a validation set (if necesary)
if val_rate > 0:
  X_train, y_train, X_val, y_val = extractValSet(X_train, polygon_ids_train, y_train, val_rate)
  #--- Computing the one-hot encoding (recomputing it for train)
  y_train_one_hot = to_categorical(y_train, n_classes)
  y_val_one_hot = to_categorical(y_val, n_classes)

if not os.path.isfile(res_file):
  if val_rate==0:
    res_mat[0,norun], res_mat[1,norun], model, model_hist, res_mat[2,norun], res_mat[3,norun] = runArchi(noarchi, X_train, y_train_one_hot, X_test, y_test_one_hot, out_model_file)
  else:
    res_mat[0,norun], res_mat[1,norun], model, model_hist, res_mat[2,norun], res_mat[3,norun] = runArchi(noarchi, X_train, y_train_one_hot, X_val, y_val_one_hot, X_test, y_test_one_hot, out_model_file)






In [ ]:

train_test_path = "/content/drive/My\ Drive/Invasives\ Research\ UMN/Remote\ Sensing\ Master/Leafy\ Spurge\ Demography/temporalCNN-master/example"

results_path = "/content/drive/My\ Drive/Invasives\ Research\ UMN/Remote\ Sensing\ Master/Leafy\ Spurge\ Demography/results"

!python3 '/content/drive/My Drive/Invasives Research UMN/Remote Sensing Master/Leafy Spurge Demography/temporalCNN-master/run_archi.py' '--sits_path' {train_test_path} '--res_path' {results_path} '--noarchi' 2

